In [1]:
# Import Dependencies
import ast
import numpy as np
import os
import pandas as pd


In [2]:
# Path to file directory and variables for the two files.
file_dir = os.path.join("..", "Data")

# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'

# The MovieLens rating data
ratings_file = f'{file_dir}/ratings_small.csv'

In [3]:
# Import Kaggle metadata, and MovieLens rating data (from Kaggle)

# Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
kaggle_metadata = pd.read_csv(kaggle_file, low_memory=False)
ratings = pd.read_csv(ratings_file)

# Import Viewer Movies and Ratings (from IMDB)
#viewer_movies = 

In [4]:
# Check DataFrames
kaggle_metadata.count()
#kaggle_metadata.head()

#ratings.count()
#ratings.head()


adult                    45466
belongs_to_collection     4494
budget                   45466
genres                   45466
homepage                  7782
id                       45466
imdb_id                  45449
original_language        45455
original_title           45466
overview                 44512
popularity               45461
poster_path              45080
production_companies     45463
production_countries     45463
release_date             45379
revenue                  45460
runtime                  45203
spoken_languages         45460
status                   45379
tagline                  20412
title                    45460
video                    45460
vote_average             45460
vote_count               45460
dtype: int64

In [5]:
# From Kaggle Data, Keep only columns: genres, id, imdb_id, title
kaggle_metadata_test = kaggle_metadata[['genres', 'id', 'imdb_id', 'title']]

kaggle_metadata_test['genres'][0]
#kaggle_metadata_test.head()

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [25]:
# TEST String Extraction:

test_string = "[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

test_list = list(ast.literal_eval(test_string))

test_list[0]['name']

replace_chars = "[']"

genres_string = str([item['name'] for item in test_list])

for char in replace_chars:
    genres_string = genres_string.replace(char, "")

genres_string

'Animation, Comedy, Family'

In [42]:
# Extract genres names from list of dicts:

replace_chars = "[']"

for index, row in kaggle_metadata_test.iterrows():
    genres_dicts_list = list(ast.literal_eval(row['genres']))
    genres_string = str([item['name'] for item in genres_dicts_list])
    
    for char in replace_chars:
        genres_string = genres_string.replace(char, "")
    
    #print(genres_string)
    row['genres_list'] = genres_string
    
    
kaggle_metadata_test.head()

,genres,id,imdb_id,title,genres_list
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,Toy Story,"Animation, Comedy, Family"
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,Jumanji,"Adventure, Fantasy, Family"
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,tt0113228,Grumpier Old Men,"Romance, Comedy"
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,tt0114885,Waiting to Exhale,"Comedy, Drama, Romance"
4,"[{'id': 35, 'name': 'Comedy'}]",11862,tt0113041,Father of the Bride Part II,Comedy


In [ ]:
# Check output
kaggle_metadata_test.head()

type(kaggle_metadata_test['genres'][0])

In [ ]:
# Drop timestamp column from ratings DataFrame
# and multiply all ratings by 2 to coincide
# with IMDB 1-10 Star Ratings
ratings.drop(['timestamp'], axis=1, inplace=True)
ratings['rating'] = ratings['rating'] * 2

ratings.head()

In [ ]:
# Create movies_df
movies_df = kaggle_metadata


In [ ]:
# Transform and merge the ratings DataFrame.
# Count Ratings
rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count() \
    .rename({'userId':'count'}, axis=1)

# Pivot Ratings Count Data
rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count() \
    .rename({'userId':'count'}, axis=1) \
    .pivot(index='movieId',columns='rating', values='count')

# Rename Ratings Count Columns
rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]

# Merge Rating Counts into movies_df
movies_with_ratings_df = pd.merge(movies_df, rating_counts, left_on='id', right_index=True, how='left')

# Fill in Missing Ratings with Zeroes
movies_with_ratings_df[rating_counts.columns] = movies_with_ratings_df[rating_counts.columns].fillna(0)



In [ ]:
# Check DataFrame
movies_with_ratings_df.count()
#movies_with_ratings_df.head()
#movies_with_ratings_df.tail()